En este script extraemos los tweets de la cuenta @ahorrandoclick1. Con este código, tan solo las respuestas correctas a las noticias, los enlaces de los tweets y algunos datos más sobre la fecha de publicación y demás información del tweet, pero no incluyen los titulares de las noticias que extraeremos en el script ObtencionTitularesClickbait usando los enlaces obtenidos en este.

In [ ]:
!pip install tweepy pandas requests beautifulsoup4

import tweepy
import pandas as pd
import requests
from bs4 import BeautifulSoup
import re
import time
from datetime import datetime
import json

class TwitterAPIMultiTokenScraper:
    def __init__(self, bearer_tokens):
        """
        Inicializar con múltiples Bearer Tokens de la API de Twitter

        Args:
            bearer_tokens (list): Lista de Bearer Tokens de la API de Twitter
        """
        self.bearer_tokens = bearer_tokens
        self.current_token_index = 0
        self.clients = [tweepy.Client(bearer_token=token) for token in bearer_tokens]

        # Headers para requests HTTP
        self.headers = {
            'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
        }
        self.session = requests.Session()
        self.session.headers.update(self.headers)

        # Para rastrear el uso de cada token
        self.token_usage = {i: 0 for i in range(len(bearer_tokens))}

    def get_current_client(self):
        """Obtener el cliente actual y rotar si es necesario"""
        return self.clients[self.current_token_index]

    def rotate_token(self):
        """Rotar al siguiente token"""
        self.current_token_index = (self.current_token_index + 1) % len(self.bearer_tokens)
        print(f"🔄 Rotando a Bearer Token #{self.current_token_index + 1}")

    def get_user_id(self, username):
        """Obtener el ID de usuario a partir del username"""
        try:
            client = self.get_current_client()
            user = client.get_user(username=username)
            if user.data:
                return user.data.id
            return None
        except Exception as e:
            print(f"Error obteniendo ID de usuario: {e}")
            return None

    def extract_urls_from_text(self, text):
        """Extraer URLs de un texto"""
        url_pattern = r'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\\(\\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
        urls = re.findall(url_pattern, text)

        # Filtrar URLs de redes sociales
        filtered_urls = []
        for url in urls:
            # Expandir URLs acortadas de t.co
            if 't.co' in url:
                try:
                    response = self.session.head(url, allow_redirects=True, timeout=10)
                    expanded_url = response.url
                    # Verificar que la URL expandida no sea de redes sociales
                    if not any(social in expanded_url.lower() for social in ['twitter.com', 'x.com', 'instagram.com', 'facebook.com']):
                        filtered_urls.append(expanded_url)
                except:
                    continue
            else:
                # URLs directas que no sean de redes sociales
                if not any(social in url.lower() for social in ['twitter.com', 'x.com', 'instagram.com', 'facebook.com']):
                    filtered_urls.append(url)

        return filtered_urls

    def get_article_title(self, url):
        """Obtener título de un artículo desde su URL"""
        try:
            response = self.session.get(url, timeout=15)
            response.raise_for_status()

            soup = BeautifulSoup(response.content, 'html.parser')

            # Intentar diferentes métodos para obtener el título
            title = None

            # Open Graph title
            og_title = soup.find('meta', property='og:title')
            if og_title and og_title.get('content'):
                title = og_title['content']

            # Twitter title
            if not title:
                twitter_title = soup.find('meta', attrs={'name': 'twitter:title'})
                if twitter_title and twitter_title.get('content'):
                    title = twitter_title['content']

            # Title tag
            if not title:
                title_tag = soup.find('title')
                if title_tag:
                    title = title_tag.text

            # H1 como último recurso
            if not title:
                h1_tag = soup.find('h1')
                if h1_tag:
                    title = h1_tag.text

            if title:
                # Limpiar el título
                title = re.sub(r'\s+', ' ', title.strip())
                # Remover información del sitio si es muy larga
                if ' - ' in title and len(title) > 100:
                    title = title.split(' - ')[0]
                elif ' | ' in title and len(title) > 100:
                    title = title.split(' | ')[0]

                return title[:200]

            return "Título no encontrado"

        except Exception as e:
            print(f"Error obteniendo título de {url}: {str(e)}")
            return "Error al obtener título"

    def get_user_tweets_batch(self, username, max_results=100, pagination_token=None):
        """
        Obtener un lote de tweets de un usuario

        Args:
            username (str): Nombre de usuario sin @
            max_results (int): Número máximo de tweets por lote
            pagination_token (str): Token para paginación
        """
        user_id = self.get_user_id(username)
        if not user_id:
            return [], None

        try:
            client = self.get_current_client()

            # Incrementar uso del token actual
            self.token_usage[self.current_token_index] += 1

            tweets = client.get_users_tweets(
                id=user_id,
                max_results=min(max_results, 100),
                tweet_fields=['created_at', 'public_metrics', 'context_annotations'],
                exclude=['retweets', 'replies'],
                pagination_token=pagination_token
            )

            if not tweets.data:
                return [], None

            # Obtener el token de paginación para la siguiente página
            next_token = tweets.meta.get('next_token') if hasattr(tweets, 'meta') and tweets.meta else None

            return tweets.data, next_token

        except tweepy.TooManyRequests:
            print(f"❌ Rate limit alcanzado para Bearer Token #{self.current_token_index + 1}")
            # Rotar al siguiente token
            self.rotate_token()
            # Esperar un poco antes de intentar con el siguiente token
            time.sleep(5)
            return [], pagination_token  # Retornar el mismo token para reintentar
        except Exception as e:
            print(f"❌ Error con Bearer Token #{self.current_token_index + 1}: {e}")
            return [], None

    def get_user_tweets_multi_token(self, username, total_tweets=500):
        """
        Obtener tweets usando múltiples tokens para superar los límites

        Args:
            username (str): Nombre de usuario sin @
            total_tweets (int): Total de tweets a obtener
        """
        print(f"🎯 Objetivo: {total_tweets} tweets de @{username}")
        print(f"📊 Bearer Tokens disponibles: {len(self.bearer_tokens)}")
        print("-" * 50)

        all_tweets_data = []
        processed_urls = set()
        pagination_token = None
        tweets_obtained = 0
        batch_size = 100  # Máximo por request en plan gratuito

        while tweets_obtained < total_tweets:
            remaining_tweets = total_tweets - tweets_obtained
            current_batch_size = min(batch_size, remaining_tweets)

            print(f"\n📥 Obteniendo lote {len(all_tweets_data)//batch_size + 1}")
            print(f"   Tweets obtenidos: {tweets_obtained}/{total_tweets}")
            print(f"   Bearer Token actual: #{self.current_token_index + 1}")

            # Obtener lote de tweets
            tweets_batch, pagination_token = self.get_user_tweets_batch(
                username, current_batch_size, pagination_token
            )

            if not tweets_batch:
                if pagination_token is None:
                    print("✅ No hay más tweets disponibles")
                    break
                else:
                    # Rotar token y continuar
                    self.rotate_token()
                    continue

            # Procesar tweets del lote
            for i, tweet in enumerate(tweets_batch):
                print(f"   Procesando tweet {i+1}/{len(tweets_batch)}: {tweet.id}")

                # Extraer URLs del texto del tweet
                urls = self.extract_urls_from_text(tweet.text)

                if urls:
                    for url in urls:
                        if url not in processed_urls:
                            processed_urls.add(url)

                            print(f"     Procesando URL: {url[:60]}...")
                            title = self.get_article_title(url)

                            if title not in ["Error al obtener título", "Título no encontrado"]:
                                tweet_data = {
                                    'tweet_id': tweet.id,
                                    'tweet_text': tweet.text,
                                    'tweet_created_at': tweet.created_at.strftime('%Y-%m-%d %H:%M:%S') if tweet.created_at else '',
                                    'url_noticia': url,
                                    'titular_noticia': title,
                                    'is_clickbait': 1,
                                    'retweet_count': tweet.public_metrics['retweet_count'] if tweet.public_metrics else 0,
                                    'like_count': tweet.public_metrics['like_count'] if tweet.public_metrics else 0,
                                    'reply_count': tweet.public_metrics['reply_count'] if tweet.public_metrics else 0,
                                    'quote_count': tweet.public_metrics['quote_count'] if tweet.public_metrics else 0,
                                    'procesado_en': datetime.now().strftime('%Y-%m-%d %H:%M:%S'),
                                    'bearer_token_usado': self.current_token_index + 1
                                }

                                all_tweets_data.append(tweet_data)
                                print(f"     ✅ Agregado: {title[:50]}...")
                            else:
                                print(f"     ❌ No se pudo obtener título")

                            # Delay para no sobrecargar servidores
                            time.sleep(1)

            tweets_obtained += len(tweets_batch)

            # Si no hay más páginas, terminar
            if not pagination_token:
                print("✅ No hay más tweets disponibles para paginar")
                break

            # Rotar token para el siguiente lote
            self.rotate_token()

            # Delay entre lotes
            time.sleep(3)

        print(f"\n✅ Procesamiento completado!")
        print(f"📊 Total de tweets procesados: {tweets_obtained}")
        print(f"📰 Total de artículos obtenidos: {len(all_tweets_data)}")

        # Mostrar uso por token
        print(f"\n📈 USO POR BEARER TOKEN:")
        for i, usage in self.token_usage.items():
            print(f"   Token #{i+1}: {usage} requests")

        return pd.DataFrame(all_tweets_data)

    def create_final_dataset(self, df):
        """Crear el dataset final con las columnas requeridas"""
        if df.empty:
            return pd.DataFrame()

        # Filtrar artículos con títulos válidos
        clean_df = df[~df['titular_noticia'].str.contains('Error|Título no encontrado', na=False)].copy()

        # Remover duplicados basados en URL
        clean_df = clean_df.drop_duplicates(subset=['url_noticia'])

        # Dataset final con solo las columnas requeridas
        final_df = clean_df[['url_noticia', 'titular_noticia', 'is_clickbait']].copy()

        return final_df

    def save_datasets(self, raw_df, final_df):
        """Guardar los datasets"""
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")

        # Dataset completo
        if not raw_df.empty:
            raw_filename = f"clickbait_raw_multitoken_{timestamp}.csv"
            raw_df.to_csv(raw_filename, index=False, encoding='utf-8')
            print(f"📁 Dataset completo guardado: {raw_filename}")

        # Dataset final
        if not final_df.empty:
            final_filename = f"clickbait_dataset_multitoken_{timestamp}.csv"
            final_df.to_csv(final_filename, index=False, encoding='utf-8')
            print(f"📁 Dataset final guardado: {final_filename}")

            return final_filename

        return None

# Función principal para usar múltiples Bearer Tokens
def create_clickbait_dataset_multi_token():
    """Función principal para crear el dataset usando múltiples Bearer Tokens"""

    print("=" * 70)
    print("🚀 CREADOR DE DATASET DE CLICKBAIT - MÚLTIPLES BEARER TOKENS")
    print("=" * 70)

    # PASO 1: Configurar Bearer Tokens
    print("\n📋 PASO 1: Configuración de múltiples Bearer Tokens")
    print("-" * 40)

    # AQUÍ DEBES PONER TUS 5 BEARER TOKENS
    BEARER_TOKENS = [
        "AAAAAAAAAAAAAAAAAAAAAO6%2BzQEAAAAAfryhIj%2BIG9KIbx3tMDhj6B0Tbm8%3DKVoE5N6ibJE5Zh8VuuCtGLkTnFe49EQRgjK2QwkuXmIAm7X4MB",
        "AAAAAAAAAAAAAAAAAAAAAFd62AEAAAAAYOBUuWTu18IxkOTfrOH50r%2BCqfc%3DbE7hjNoebVw9AvKpYDSr2AA4ooNaQO9Vua51nTttslJjndLCQv",
        "AAAAAAAAAAAAAAAAAAAAALN62AEAAAAArziUSqloB0%2BPSmM5tUuyp4HU%2BGU%3DL0m0vYE2iBgqABL5wUH1KcBlJbBOnhmWEPSYlydgY8eFJOBrCr",
        "AAAAAAAAAAAAAAAAAAAAAOF62AEAAAAA5aF%2F%2BM%2FgwXyCEDjYjtiCLV8ODqA%3DhicC7ZhR5SDJ6fl4vs6ztRmlMhWLDmftmv2vzkpE1HkUMKu1R2",
        "AAAAAAAAAAAAAAAAAAAAAOZ62AEAAAAAddltHTioEAGrHkvZlMuNwvuv4tg%3DHl6OodDftl5lFhC9vlykmg4f1H93TkeQ7L5yuN6evXmb94EPXP"
    ]

    # Verificar configuración
    if any(token.startswith("YOUR_BEARER_TOKEN") for token in BEARER_TOKENS):
        print("❌ CONFIGURACIÓN REQUERIDA:")
        print("1. Ve a https://developer.twitter.com/en/portal/dashboard")
        print("2. Crea 5 aplicaciones diferentes (o usa 5 cuentas)")
        print("3. Copia los 5 Bearer Tokens")
        print("4. Reemplaza cada 'YOUR_BEARER_TOKEN_X_HERE' en la lista")
        print("5. Ejecuta de nuevo")
        print("\n💡 Cada Bearer Token se ve así:")
        print("   AAAAAAAAAAAAAAAAAAAAAMLheAAAAAAA0%2BuSeid%2BULvsea4JtiGRiSDSJSI%3DEUifiRBkKG5E2XzMDjRfl76ZC9Ub0wnz4XsNiRVBChTYbJcE3F")
        return None

    # PASO 2: Crear el scraper multi-token
    scraper = TwitterAPIMultiTokenScraper(BEARER_TOKENS)

    # PASO 3: Configurar parámetros
    username = "ahorrandoclick1"  # Sin el @
    total_tweets = 500  # Total deseado

    print(f"\n📊 PASO 2: Configuración del scraping")
    print(f"Usuario objetivo: @{username}")
    print(f"Total de tweets deseados: {total_tweets}")
    print(f"Bearer Tokens configurados: {len(BEARER_TOKENS)}")
    print("-" * 40)

    # PASO 4: Obtener tweets usando múltiples tokens
    raw_df = scraper.get_user_tweets_multi_token(username, total_tweets)

    if raw_df.empty:
        print("❌ No se obtuvieron datos")
        return None

    # PASO 5: Crear dataset final
    print(f"\n📊 PASO 3: Procesamiento de datos")
    print("-" * 40)

    final_df = scraper.create_final_dataset(raw_df)

    # PASO 6: Mostrar estadísticas detalladas
    print(f"\n📈 ESTADÍSTICAS FINALES:")
    print(f"Tweets únicos procesados: {len(raw_df)}")
    print(f"URLs únicas obtenidas: {len(final_df)}")
    if not final_df.empty:
        print(f"Promedio de caracteres en títulos: {final_df['titular_noticia'].str.len().mean():.1f}")
        print(f"Título más largo: {final_df['titular_noticia'].str.len().max()} caracteres")
        print(f"Título más corto: {final_df['titular_noticia'].str.len().min()} caracteres")

    # Análisis por Bearer Token
    if not raw_df.empty:
        print(f"\n📊 ANÁLISIS POR BEARER TOKEN:")
        token_stats = raw_df.groupby('bearer_token_usado').size()
        for token_num, count in token_stats.items():
            print(f"   Bearer Token #{token_num}: {count} artículos obtenidos")

    # PASO 7: Guardar archivos
    print(f"\n💾 PASO 4: Guardando archivos")
    print("-" * 40)

    filename = scraper.save_datasets(raw_df, final_df)

    # PASO 8: Mostrar muestra
    if not final_df.empty:
        print(f"\n📋 MUESTRA DEL DATASET FINAL:")
        print("=" * 70)
        for i, row in final_df.head(5).iterrows():
            print(f"{i+1}. {row['titular_noticia']}")
            print(f"   URL: {row['url_noticia'][:60]}...")
            print(f"   Clickbait: {row['is_clickbait']}")
            print("-" * 50)

    # PASO 9: Información sobre límites y recomendaciones
    print(f"\n⚠️  INFORMACIÓN IMPORTANTE:")
    print("✅ Has usado múltiples Bearer Tokens para obtener más datos")
    print("✅ Cada token tiene su propio límite de 100 tweets/mes")
    print("✅ Los tokens se rotan automáticamente para maximizar la obtención")
    print("💡 Para obtener aún más datos, considera:")
    print("   - Usar más Bearer Tokens (puedes añadir más a la lista)")
    print("   - Actualizar a planes Basic ($100/mes por token)")
    print("   - Ejecutar el script en diferentes momentos del mes")

    return final_df

# Ejecutar automáticamente
dataset = create_clickbait_dataset_multi_token()

🎯 OPCIONES DISPONIBLES:
1. create_clickbait_dataset_multi_token() - Crear dataset con múltiples tokens
2. test_all_bearer_tokens() - Probar todos los Bearer Tokens

▶️  Ejecutando creación de dataset con múltiples tokens...
🚀 CREADOR DE DATASET DE CLICKBAIT - MÚLTIPLES BEARER TOKENS

📋 PASO 1: Configuración de múltiples Bearer Tokens
----------------------------------------

📊 PASO 2: Configuración del scraping
Usuario objetivo: @ahorrandoclick1
Total de tweets deseados: 500
Bearer Tokens configurados: 5
----------------------------------------
🎯 Objetivo: 500 tweets de @ahorrandoclick1
📊 Bearer Tokens disponibles: 5
--------------------------------------------------

📥 Obteniendo lote 1
   Tweets obtenidos: 0/500
   Bearer Token actual: #1
   Procesando tweet 1/100: 1928376813148897524
     Procesando URL: https://t.co/akeHX8Radv...
     ✅ Agregado: https://twitter.com/sport/status/19283720565693157...
   Procesando tweet 2/100: 1928376585255604600
     Procesando URL: https://t.co/pw